# Multimodal

## Setup

### Imports

In [2]:
# Imports
import os
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, pipeline
from torch.utils.data import Dataset, DataLoader
from src.utils import *
from src.models import *
from src.process_reports import *
from src.train import train_mm_stil, kfold_cv

%load_ext autoreload
%autoreload 2

### Set seed & device

In [3]:
set_seed(42)      
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Extract text feats from reports

In [3]:
%%script false --no-raise-error
# load biobert model & tokenizer
tokenizer = AutoTokenizer.from_pretrained(''dmis-lab/biobert-large-cased-v1.1-mnli)
lm = AutoModel.from_pretrained('dmis-lab/biobert-large-cased-v1.1-mnli')
data_dir = '/mnt/disks/ext/data/gdc/tcga/brca'
output_dir = 'data/report_feats'
extract_text_features(lm, tokenizer, data_dir)

## Task: Predict sTILs from WSIs & reports

### Set hparams

In [3]:
bsz = 16 # batch size for dataloaders
img_channels_in = 2048  # emb dim of wsi feats
text_channels_in = 1024 # emb dim of report feats
hidden_dim_mlp = 16 # hidden layer dim of mlp
num_classes = 10 # number of stil levels

### Load data

In [4]:
# Create the dataset
root_dir = './'
data_file = 'data/stils/data_stils.csv'
use_rand_splits = False # use random splits or predefined splits
train_data = MMDataset(root_dir, data_file, 'train', use_rand_splits)
val_data = MMDataset(root_dir, data_file, 'val', use_rand_splits)
test_data = MMDataset(root_dir, data_file, 'test', use_rand_splits)

print(f'size of train set: {len(train_data)}, val set: {len(val_data)}, test set: {len(test_data)}')

# Create the dataloaders
train_loader = DataLoader(train_data, batch_size=bsz, shuffle=True, num_workers=12, collate_fn=MMDataset.mm_collate_fn)
val_loader = DataLoader(val_data, batch_size=bsz, shuffle=False, num_workers=12, collate_fn=MMDataset.mm_collate_fn)
test_loader = DataLoader(test_data, batch_size=bsz, shuffle=False, num_workers=12, collate_fn=MMDataset.mm_collate_fn)

size of train set: 468, val set: 58, test set: 170


### Train & eval

In [ ]:
# %%script false --no-raise-error
# init model (one of: MLPSTILClassifier, MLPSTILRegressor, Attention1DSTILClassifier, Attention1DSTILRegressor, Attention2DSTILClassifier, Attention2DSTILRegressor) 
model = Attention1DSTILRegressor(mode='image')

# set training args
num_epochs = 50
resume_ckpt = None
args = {'num_epochs': num_epochs, 'ckpt_name': 'ckpt_best_img'}

# train model
model, trainer = train_mm_stil(model, train_loader, val_loader, args)

# evaluate the trained model on the test set
trainer.test(model, test_loader)

### K-fold CV

In [ ]:
# %%script false --no-raise-error
# run k-fold CV
dataset = MMDataset()

model_class = Attention1DSTILRegressor
model_args = {'mode': 'image'}
train_args = {'k': 5, 'num_epochs': 50, 'ckpt_name': 'ckpt_best_img_kfold_cv'}

res_kfold_cv = kfold_cv(model_class, dataset, model_args, train_args)
metrics = ['test_loss', 'test_corr', 'test_r2']
avg_res = {k: np.mean([res[k] for res in res_kfold_cv]).round(3) for k in metrics}
print(f"avg res over {train_args['k']} folds: {avg_res}")

## Task: Predict cancer subtypes from multimodal data

### Annotate subtype & grade from path reports

In [6]:
lm_name = 'gpt-3.5-turbo'

# sample_report_path = 'data/reports_distilled/TCGA-WT-AB41.txt'
reports_dir = 'data/reports_distilled'
# create prompt
# prompt = create_zs_prompt(sample_report_path)

# args for generation
gen_args = {'max_tokens': 200}

# out = gen_subtype_grade_zs(lm_name, prompt, api='openai', args=gen_args)
df_res = classify_reports_zs(lm_name, reports_dir, api='openai', args=gen_args)
df_res.head(10)

  0%|          | 0/1068 [00:00<?, ?it/s]

 21%|██        | 219/1068 [03:57<15:56,  1.13s/it]

error parsing output for TCGA-C8-A27A.txt...


 88%|████████▊ | 939/1068 [16:19<02:11,  1.02s/it]

HTTP error: 502 Server Error: Bad Gateway for url: https://api.openai.com/v1/chat/completions, retrying...


 99%|█████████▉| 1055/1068 [28:29<00:13,  1.06s/it]  

HTTP error: 502 Server Error: Bad Gateway for url: https://api.openai.com/v1/chat/completions, retrying...


100%|██████████| 1068/1068 [38:51<00:00,  2.18s/it] 


,case_id,region,localization,grade
13,TCGA-3C-AAAU,Lobular,Invasive,3
952,TCGA-3C-AALI,Ductal,Invasive,2
458,TCGA-3C-AALJ,Ductal,Invasive,3
701,TCGA-3C-AALK,ductal,invasive,NA
568,TCGA-4H-AAAK,Lobular,Invasive,2
420,TCGA-5L-AAT0,lobular,invasive,1
878,TCGA-5L-AAT1,lobular,invasive,1
341,TCGA-5T-A9QA,"Ductal (as the report mentions ""invasive ducta...","Invasive (as the report mentions ""invasive car...","3 (as the report mentions ""Histologic Grade"
1051,TCGA-A1-A0SB,other (adenoid cystic carcinoma),invasive,1
371,TCGA-A1-A0SD,Ductal,Invasive,Grade 2
